In [1]:
import logging
import os
import gensim
import smart_open
import  collections
import random
import warnings 
import logging
warnings.filterwarnings(action='ignore',category=UserWarning,module='gensim')


In [2]:
#Set up logging configurations  
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence

class LabeledLineSentence(object):
    def __init__(self, filename):
        self.filename = filename
    def __iter__(self):
        with open(self.filename, 'r') as f:
            for uid, line in enumerate(f):
                
                yield LabeledSentence(words=line.split(), tags=['TXT_%s' % uid])

sentences = LabeledLineSentence("/home/fatemeh/Documents/python/dnn-understanding/code/wiki.en.text")

print(sentences)

In [4]:
model = gensim.models.doc2vec.Doc2Vec(size=300, min_count=5, iter=20)
#(size=400, min_count=3, iter=55)    

model.build_vocab(sentences)

print("vocab done")

%time model.train(sentences, total_examples=model.corpus_count)

print("train")

model.save("doc2vec_wiki_D_300.txt")

In [8]:
model.infer_vector(['only', 'you', 'can'])

array([ -2.86672078e-03,  -4.83214855e-03,   5.82147762e-03,
         2.83221155e-03,  -5.38526615e-03,   7.35091418e-03,
         6.48303702e-03,  -5.22502000e-03,  -2.23284075e-03,
         6.11268636e-03,   6.62072888e-03,   2.67294561e-03,
        -1.11131044e-03,   1.73574768e-03,  -2.84287869e-03,
         5.67549141e-03,  -5.10061812e-03,   2.49748467e-03,
        -6.90249214e-03,   5.90707874e-03,  -4.88218758e-03,
        -4.11457819e-04,   9.15659685e-03,   7.03204796e-03,
         7.10814865e-03,  -2.53660418e-03,   6.21377316e-04,
        -5.90374414e-03,   4.25973628e-03,   9.17429011e-03,
        -1.52549567e-03,   5.44566289e-03,   6.38154510e-04,
        -3.61815980e-03,   4.54226928e-03,   3.21548781e-03,
         5.47764730e-03,   3.66133492e-04,  -9.37171094e-03,
        -5.15841786e-03,   4.97694500e-03,  -3.73700773e-03,
         7.08529819e-03,   3.24116081e-05,  -1.44633159e-04,
        -2.43902300e-03,   7.25570740e-03,  -8.71502608e-03,
         6.54574996e-03,

In [9]:
print (model.most_similar("fires"))

[('et', 0.9889155626296997), ('leading', 0.9881570339202881), ('those', 0.9879174828529358), ('Canada', 0.9876338243484497), ('Liberal', 0.9875488877296448), ('Grove', 0.987313985824585), ('matches', 0.9872931838035583), ('slogan', 0.9872443079948425), ('Good', 0.9871824979782104), ('management', 0.9871189594268799)]


In [10]:
print (model["fires"])

[ 0.01571665 -0.03477355  0.03999928 -0.00027907  0.08313894 -0.03727892
  0.03461172 -0.03189727 -0.07338314  0.04325727 -0.04484431  0.03648184
  0.05015948  0.04913468 -0.03347735  0.02325353 -0.03105309 -0.03227264
  0.06165224 -0.06730888 -0.05078313 -0.06425183  0.05975012  0.05655986
  0.00665964  0.01426202  0.04680209 -0.06289101  0.03641322  0.05236815
 -0.04623964 -0.03227931  0.03402216  0.05000139  0.05581727  0.02511236
  0.02659604  0.03144038 -0.03323868 -0.03595668 -0.02675604 -0.01941067
  0.01402176  0.00726173  0.03886427 -0.01086277  0.06725353 -0.03409914
 -0.02325037 -0.07153598]


In [11]:
print (model.most_similar('election', topn=10))

[('Atlanta', 0.9987221956253052), ('persons', 0.9986709952354431), ('candidates', 0.998654842376709), ('County', 0.9986493587493896), ('county', 0.9986488819122314), ('city', 0.9986240267753601), ('Judge', 0.9986098408699036), ('votes', 0.9985962510108948), ('said', 0.9985662698745728), ('yesterday', 0.998558521270752)]


In [13]:
model.most_similar('absurdity', topn=3)

[('illustrative', 0.4938625693321228),
 ('host', 0.4724528193473816),
 ('Rather', 0.4650704264640808)]

In [14]:
model.most_similar('manner', topn=4)

[('county', 0.9974332451820374),
 ('into', 0.9973833560943604),
 ('basketball', 0.9973552823066711),
 ("they're", 0.9973540306091309)]

In [16]:
def print_accuracy(model, questions_file):
    print('Evaluating...\n')
    acc = model.accuracy(questions_file)

    sem_correct = sum((len(acc[i]['correct']) for i in range(5)))
    sem_total = sum((len(acc[i]['correct']) + len(acc[i]['incorrect'])) for i in range(5))
    sem_acc = 100*float(sem_correct)/sem_total
    print('\nSemantic: {:d}/{:d}, Accuracy: {:.2f}%'.format(sem_correct, sem_total, sem_acc))
    
    syn_correct = sum((len(acc[i]['correct']) for i in range(5, len(acc)-1)))
    syn_total = sum((len(acc[i]['correct']) + len(acc[i]['incorrect'])) for i in range(5,len(acc)-1))
    syn_acc = 100*float(syn_correct)/syn_total
    print('Syntactic: {:d}/{:d}, Accuracy: {:.2f}%\n'.format(syn_correct, syn_total, syn_acc))
    return (sem_acc, syn_acc)

In [17]:
word_analogies_file = '/home/fatemeh/Documents/python/dnn-understanding/code/questions-words.txt'
accuracies = []

Total: 56 out of 506, Correct: 0.00%, Incorrect: 100.00%


In [ ]:
print('Accuracy for word2vec:')
accuracies.append(print_accuracy(model,word_analogies_file))

In [ ]:
model.evaluate_word_pairs('/home/fatemeh/Documents/python/dnn-understanding/code/accuracy/eval/eval.tab',case_insensitive=True,delimiter='\t', restrict_vocab=30000, dummy4unknown=False)